In [1]:
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image
import torch
from PIL import Image, ImageOps, ImageFilter

# Setup Pipeline
pipe = AutoPipelineForInpainting.from_pretrained(
    "diffusers/stable-diffusion-xl-1.0-inpainting-0.1",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")
pipe.enable_model_cpu_offload()

# Load Inputs
img_path = "output_masks/unnamed/original_image.jpg"
mask_path = "output_masks/unnamed/pants_mask.png"

original_image = load_image(img_path).resize((1024, 1024))
mask_input = load_image(mask_path).resize((1024, 1024))


mask_image = mask_input.convert("L")
mask_image = mask_image.point(lambda x: 255 if x > 128 else 0)


mask_image = mask_image.filter(ImageFilter.MaxFilter(11)) 

debug_overlay = original_image.copy()
red_layer = Image.new("RGB", original_image.size, (255, 0, 0))
debug_overlay.paste(red_layer, (0, 0), mask_image)

debug_overlay.show() 

# ---------------------------------------------

# Generate
prompt = "simple green jeans, high quality, made of cotton"

output = pipe(
    prompt=prompt,
    image=original_image,
    mask_image=mask_image,
    num_inference_steps=30,
    strength=1.0,
    guidance_scale=7.5
).images[0]

output.show()
output.save("fixed_clothing_result.png")

c:\Users\User\miniconda3\envs\flux-clean\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 30/30 [07:25<00:00, 14.86s/it]
